In [ ]:
#THIS FUNCTION IS FOR RUNNING WITH SLURM JOB ARRAY
#(SPLITS UP JOB_ARRAY BELOW INTO EVEN MORE TASKS)
def StartSlurmJobArray(num_jobs,num_slurm_jobs, ISRUN):
    job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    if job_id==0: job_id=2
    if ISRUN==False:
        start_job=1;end_job=num_jobs+1
        return start_job,end_job
    total_elements=num_jobs #total num of variables

    job_range = total_elements // num_slurm_jobs  # Base size for each chunk
    remaining = total_elements % num_slurm_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id, num_slurm_jobs):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_slurm_jobs - 1: 
            end_job = total_elements 
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_slurm_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()
    # if sbatch==True:
        
    start_job, end_job = get_job_range(job_id, num_slurm_jobs)
    index_adjust=start_job
    # print(f'start_job = {start_job}, end_job = {end_job}')
    if start_job==0: start_job=1
    if end_job==total_elements: end_job+=1
    return start_job,end_job

# job_id=1
# [start_slurm_job,end_slurm_job,slurm_index_adjust]=StartSlurmJobArray(num_jobs,num_slurm_jobs,ISRUN)
# parcel=parcel1.isel(xh=slice(start_job,end_job))

In [ ]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import matplotlib.lines as mlines
import xarray as xr
import os; import time
import pickle
import h5py

In [ ]:
#MAIN DIRECTORIES
mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
scratchDirectory='/home/air673/koa_scratch/'
codeDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Project_Algorithms/Tracking_Algorithms'

In [ ]:
#LOADING DATA
def GetDataDirectories(simulationNumber):
    if simulationNumber == 1:
        Directory=os.path.join(mainDirectory,'Model/cm1r20.3/run')
        res='1km'; t_res='5min'; Np_str='1e6'; Nz_str='34'
    elif simulationNumber == 2:
        Directory=scratchDirectory
        res='1km'; t_res='1min'; Np_str='50e6'; Nz_str='95'
    elif simulationNumber == 3:
        Directory=scratchDirectory
        res='250m'; t_res='1min'; Np_str='50e6'; Nz_str='95'
        
    dataDirectory = os.path.join(Directory, f"cm1out_{res}_{t_res}_{Nz_str}nz.nc")
    parcelDirectory = os.path.join(Directory,f"cm1out_pdata_{res}_{t_res}_{Np_str}np.nc")
    return dataDirectory, parcelDirectory, res,t_res,Np_str,Nz_str
    
def GetData(dataDirectory, parcelDirectory):
    dataNC = xr.open_dataset(dataDirectory, decode_timedelta=True) 
    parcelNC = xr.open_dataset(parcelDirectory, decode_timedelta=True) 
    return dataNC,parcelNC

def SubsetDataVars(dataNC):
    varList = ["thflux", "qvflux", "tsk", "cape", 
               "cin", "lcl", "lfc", "th",
               "prs", "rho", "qv", "qc",
               "qr", "qi", "qs","qg", 
               "buoyancy", "uinterp", "vinterp", "winterp",]
    
    varList += ["ptb_hadv", "ptb_vadv", "ptb_hidiff", "ptb_vidiff",
                "ptb_hturb", "ptb_vturb", "ptb_mp", "ptb_rdamp", 
                "ptb_rad", "ptb_div", "ptb_diss",]
    
    varList += ["qvb_hadv", "qvb_vadv", "qvb_hidiff", "qvb_vidiff", 
                "qvb_hturb", "qvb_vturb", "qvb_mp",]
    
    varList += ["wb_hadv", "wb_vadv", "wb_hidiff", "wb_vidiff",
                "wb_hturb", "wb_vturb", "wb_pgrad", "wb_rdamp", "wb_buoy",]

    return dataNC[varList]

[dataDirectory,parcelDirectory, res,t_res,Np_str,Nz_str] = GetDataDirectories(simulationNumber=1)
[data1,parcel1] = GetData(dataDirectory, parcelDirectory)

In [ ]:
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
job_array=False;index_adjust=0
ocean_fraction=2/8
start_time = time.time();

In [ ]:
#########################################

In [ ]:
import psutil
def print_memory_usage(note=""):
    process = psutil.Process(os.getpid())
    mem = process.memory_info().rss / 1024 ** 2  # in MB
    print(f"[{note}] Memory usage: {mem:.2f} MB")
print_memory_usage("Current Memory")

In [ ]:
times=data1['time'].values/(1e9 * 60); times=times.astype(float);
minutes=1/times[1] #1 / minutes per timestep = timesteps per minute
kms=np.argmax(data1['xh'].values-data1['xh'][0].values >= 1)

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(PlottingFunctions, inspect.isfunction)]
# functions

In [ ]:
#READING BACK IN
########################
def LoadFinalData(in_file):
    dict = {}
    with h5py.File(in_file, 'r') as f:
        for key in f.keys():
            dict[key] = f[key][:]
    return dict

def LoadAllCloudBase():
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
    in_file = dir2 + f"all_cloudbase_{res}_{t_res}_{Np_str}.pkl"
    with open(in_file, 'rb') as f:
        all_cloudbase = pickle.load(f)
    return(all_cloudbase)
min_all_cloudbase=np.nanmin(LoadAllCloudBase())
print(f"Minimum Cloudbase is: {min_all_cloudbase*1000:.0f} m")

def LoadMeanLFC():
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
    in_file = dir2 + f"MeanLFC_{res}_{t_res}_{Np_str}.pkl"
    with open(in_file, 'rb') as f:
        MeanLFC = pickle.load(f)
    return MeanLFC
MeanLFC=LoadMeanLFC()
print(f"Mean LFC is: {MeanLFC:.0f} m\n")

dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
in_file=dir2+f"parcel_tracking_SUBSET_{res}_{t_res}_{Np_str}.h5"
final_dict=LoadFinalData(in_file)


#DYNAMICALLY CREATING VARIABLES
for key, value in final_dict.items():
    globals()[key] = value

# #DYNAMICALLY PRINTING VARIABLE SIZES
# for key in final_dict:
#     print(f"{key} has {final_dict[key].shape[0]} parcels")

# PRINTING VARIABLE SIZES (ONE BY ONE)
print(f'ALL: {len(CL_ALL_out_arr)} CL parcels and {len(nonCL_ALL_out_arr)} nonCL parcels')
print(f'SHALLOW: {len(CL_SHALLOW_out_arr)} CL parcels and {len(nonCL_SHALLOW_out_arr)} nonCL parcels')
print(f'DEEP: {len(CL_DEEP_out_arr)} CL parcels and {len(nonCL_DEEP_out_arr)} nonCL parcels')
print('\n')
print(f'ALL: {len(SBZ_ALL_out_arr)} SBZ parcels and {len(nonSBZ_ALL_out_arr)} nonSBZ parcels')
print(f'SHALLOW: {len(SBZ_SHALLOW_out_arr)} SBZ parcels and {len(nonSBZ_SHALLOW_out_arr)} nonSBZ parcels')
print(f'DEEP: {len(SBZ_DEEP_out_arr)} SBZ parcels and {len(nonSBZ_DEEP_out_arr)} nonSBZ parcels')
print('\n')
print(f'ALL: {len(ColdPool_ALL_out_arr)} ColdPool parcels')
print(f'SHALLOW: {len(ColdPool_SHALLOW_out_arr)} ColdPool parcels')
print(f'DEEP: {len(ColdPool_DEEP_out_arr)} ColdPool parcels')


def apply_job_array_filter(start_job, end_job):
    # print("APPLYING JOB ARRAY")

    def job_filter(arr):
        return arr[(arr[:, 0] >= start_job) & (arr[:, 0] < end_job)]

    target_names = [
        'CL_ALL_out_arr', 'nonCL_ALL_out_arr',
        'CL_SHALLOW_out_arr', 'nonCL_SHALLOW_out_arr',
        'CL_DEEP_out_arr', 'nonCL_DEEP_out_arr',
        'SBZ_ALL_out_arr', 'nonSBZ_ALL_out_arr',
        'SBZ_SHALLOW_out_arr', 'nonSBZ_SHALLOW_out_arr',
        'SBZ_DEEP_out_arr', 'nonSBZ_DEEP_out_arr',
        'ColdPool_ALL_out_arr', 'ColdPool_SHALLOW_out_arr', 'ColdPool_DEEP_out_arr'
    ]

    for name in target_names:
        globals()[name+'_2'] = job_filter(globals()[name])

In [ ]:
##############################################
#SETUP

In [ ]:
PROCESSING=False
PROCESSING=True

In [ ]:
def MakeDictionary(**vars):
    return vars
    
def GetData1(start_job,end_job):
    dir2=dir+'Project_Algorithms/Lagrangian_Arrays/OUTPUT/'
    in_file=dir2+f'lagrangian_binary_array_{res}_{t_res}_{Np_str}.h5'
    
    var_names = ['W', 'QCQI']
    data_dict = make_data_dict(in_file,var_names,read_type,start_job,end_job)
    W, QCQI = (data_dict[k] for k in var_names)
    
    # #Making Time Matrix
    # rows, cols = A.shape[0], A.shape[1]
    # T = np.arange(rows).reshape(-1, 1) * np.ones((1, cols), dtype=int)
    
    dir2=dir+'Project_Algorithms/Lagrangian_Arrays/OUTPUT/'
    in_file=dir2+f'lagrangian_VARS_array_{res}_{t_res}_{Np_str}.h5'
    
    # var_names = ['QV','TH','TH_V','TH_E','RH_VAPOR','RH_ICE','BUOYANCY_CM1','BUOYANCY_FULL','BUOYANCY_FULL_EACH_T','HMC']
    var_names = ['QV','RH_VAPOR','TH_V','TH_E','MSE','HMC','VMF_G','VMF_C']
    data_dict = make_data_dict(in_file,var_names,read_type,start_job,end_job)
    QV, RH_VAPOR, TH_V, TH_E, MSE, HMC, VMF_G, VMF_C = [data_dict[k] for k in var_names] #, QI, QR

    VARs=MakeDictionary(W=W, QCQI=QCQI, QV=QV, RH_VAPOR=RH_VAPOR,
                        TH_V=TH_V, TH_E=TH_E, MSE=MSE, HMC=HMC, VMF_G=VMF_G, VMF_C=VMF_C)
    return VARs

def GetData2(start_job,end_job):    
    dir2=dir+'Project_Algorithms/Lagrangian_Arrays/OUTPUT/'
    if PROCESSING==True:
        in_file=dir2+f'lagrangian_ED_VARS_array_{res}_{t_res}_{Np_str}.h5'
    elif PROCESSING==False:
        in_file=dir2+f'lagrangian_ED_VARS_array_{res}_{t_res}_{Np_str}_ORIGINAL.h5'
    
    var_names = ['E_G','E_C','D_C','D_G']
    data_dict = make_data_dict(in_file,var_names,read_type,start_job,end_job)
    E_G, E_C, D_C, D_G = (data_dict[k] for k in var_names)

    NET_G, NET_C = E_G-D_G, E_C-D_C

    VARs=MakeDictionary(E_G=E_G, E_C=E_C, D_C=D_C, D_G=D_G, NET_G=NET_G, NET_C=NET_C)
    return VARs

def GetData3(start_job,end_job):    
    dir3=dir+'Project_Algorithms/Lagrangian_Arrays/OUTPUT/'
    if PROCESSING==True:
        in_file=dir3+f'lagrangian_Combined_ED_VARS_array_{res}_{t_res}_{Np_str}.h5' 
    elif PROCESSING==False:
        in_file=dir3+f'lagrangian_Combined_ED_VARS_array_{res}_{t_res}_{Np_str}_ORIGINAL.h5' 
        
    var_names = ["C_TO_G_E","G_TO_C_E","C_TO_G_D","G_TO_C_D"]
    data_dict = make_data_dict(in_file,var_names,read_type,start_job,end_job)
    [C_TO_G_E, G_TO_C_E, C_TO_G_D, G_TO_C_D] = (data_dict[k] for k in var_names)

    VARs=MakeDictionary(C_TO_G_E=C_TO_G_E, G_TO_C_E=G_TO_C_E, C_TO_G_D=C_TO_G_D, G_TO_C_D=G_TO_C_D)
    return VARs


def GetData4(start_job,end_job):    
    dir3=dir+'Project_Algorithms/Lagrangian_Arrays/OUTPUT/'
    if PROCESSING==True:
        in_file=dir3+f'lagrangian_BUOYANCY_VARS_array_{res}_{t_res}_{Np_str}.h5' 
    elif PROCESSING==False:
        in_file=dir3+f'lagrangian_BUOYANCY_VARS_array_{res}_{t_res}_{Np_str}_ORIGINAL.h5' 
        
    var_names = ['BUOYANCY','TH_TERM','RV_TERM','RL_TERM']
    data_dict = make_data_dict(in_file,var_names,read_type,start_job,end_job)
    [BUOYANCY,TH_TERM,RV_TERM,RL_TERM] = (data_dict[k] for k in var_names)

    VARs=MakeDictionary(BUOYANCY=BUOYANCY,TH_TERM=TH_TERM,RV_TERM=RV_TERM,RL_TERM=RL_TERM)
    return VARs

In [ ]:
################################
#DATA SETUP
################################
#*#*
data_type="Tracked_Properties" 
# data_type="Tracked_Entrainment_VMF"
# data_type="Tracked_Combined_Entrainment" 

if data_type=="Tracked_Properties":
    GetData=GetData1
    variables = ["W","QCQI","QV","RH_VAPOR","TH_V","TH_E","MSE","HMC",'VMF_G','VMF_C']
elif data_type=="Tracked_Entrainment_VMF":
    GetData=GetData2
    variables = ["VMF_C","VMF_G","E_G","E_C","D_C","D_G","NET_G","NET_C"]
elif data_type=="Tracked_Combined_Entrainment":
    GetData=GetData3
    variables = ["C_TO_G_E","G_TO_C_E","C_TO_G_D","G_TO_C_D"]
##############################

In [ ]:
################################
#JOB ARRAY SETUP
################################
#*#*
# how many total jobs are being run? i.e. array=1-100 ==> num_jobs=100
if Np_str=='1e6':
    num_jobs=60 #1M parcels
    num_slurm_jobs=10
if Np_str=='50e6':
    num_jobs=200 #50M parcels
    num_slurm_jobs=60
##############################

In [ ]:
##############################################
#DATA LOADING FUNCTIONS

In [ ]:
#JOB ARRAY SETUP
def StartJobArray(job_id,num_jobs):
    total_elements=len(parcel1['xh']) #total num of variables

    if num_jobs >= total_elements:
        raise ValueError("Number of jobs cannot be greater than or equal to total elements.")
    
    job_range = total_elements // num_jobs  # Base size for each chunk
    remaining = total_elements % num_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id, num_jobs):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_jobs - 1: 
            end_job = total_elements #- 1
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()

    # if sbatch==True:
    #     job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    #     if job_id==0: job_id=1
        
    start_job, end_job = get_job_range(job_id, num_jobs)
    index_adjust=start_job
    # print(f'start_job = {start_job}, end_job = {end_job}')
    return start_job,end_job,index_adjust

# job_id=1
# [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)
# parcel=parcel1.isel(xh=slice(start_job,end_job))

In [ ]:
# Reading Back Data Later
##############
def make_data_dict(in_file,var_names,read_type,start_job,end_job):
    if read_type=='h5py':
        with h5py.File(in_file, 'r') as f:
            data_dict = {var_name: f[var_name][:,start_job:end_job] for var_name in var_names}
            
    elif read_type=='xarray':
        in_data = xr.open_dataset(
            in_file,
            engine='h5netcdf',
            phony_dims='sort',
            chunks={'phony_dim_0': 100, 'phony_dim_1': 1_000_000} 
        )
        data_dict = {k: in_data[k][:,start_job:end_job].compute().data for k in var_names}
    return data_dict

# read_type='xarray'
read_type='h5py'

In [ ]:
def GetSpatialData(start_job,end_job):
    dir2=dir+'Project_Algorithms/Lagrangian_Arrays/OUTPUT/'
    in_file=dir2+f'lagrangian_binary_array_{res}_{t_res}_{Np_str}.h5'
    
    var_names = ['Z', 'Y', 'X', 'z']
    data_dict = make_data_dict(in_file,var_names,read_type,start_job,end_job)
    Z, Y, X, parcel_z = (data_dict[k] for k in var_names)
    
    # #Making Time Matrix
    # rows, cols = A.shape[0], A.shape[1]
    # T = np.arange(rows).reshape(-1, 1) * np.ones((1, cols), dtype=int)
    
    return Z,Y,X,parcel_z

In [ ]:
############################################################################################

In [ ]:
##############################################
#MAKE PROFILES FUNCTIONS

In [ ]:
def SubsetTimeOverlap(Array, ts_window):
    t1 = Array[:, 1]
    t2 = Array[:, 2]
    w1, w2 = ts_window[0], ts_window[-1]
    keep = (t2 >= w1) & (t1 <= w2)
    return Array[keep]
def build_ZYX(ps, t1s, t2s, Z, Y, X, index_adjust):
    rows = []
    for p, t1, t2 in zip(ps.astype(int), t1s.astype(int), t2s.astype(int)):
        t_range = np.arange(t1, t2 + 1)
        rows.append(np.vstack([
            np.full_like(t_range, p),
            t_range,
            Z[t_range, p - index_adjust],
            Y[t_range, p - index_adjust],
            X[t_range, p - index_adjust]
        ]))
    return np.hstack(rows) if rows else np.empty((5, 0))
def match_p_values(A, B, tol=1e-8):
    """
    Find parcel IDs (p values) in A that have a matching column in B.
    First row (p) is matched exactly; rows 1..4 are matched with tolerance.

    Args:
        A: np.ndarray of shape (5, NA) [p, t, z, y, x]
        B: np.ndarray of shape (5, NB) [p, t, z, y, x]
        tol: tolerance for np.isclose on rows 1..4

    Returns:
        p_values: array of parcel IDs from A that match a column in B
        i_idx: indices in A of matching columns
        j_idx: indices in B of matching columns
    """
    if A.size == 0 or B.size == 0:
        return np.array([])

    # exact match on p
    eq = np.empty((5, A.shape[1], B.shape[1]), dtype=bool)
    eq[0] = (A[0, :, None] == B[0, None, :])

    # tolerant match on t,z,y,x
    eq[1:] = np.isclose(A[1:, :, None], B[1:, None, :], atol=tol, rtol=0)

    # find all column pairs that match entirely
    pairwise = eq.all(axis=0)
    i_idx, j_idx = np.where(pairwise) #columns in A and B respectively

    # extract p values from A
    p_values = A[0, i_idx]

    return p_values
def _init_empty(dtype):
    return np.empty((0, 3), dtype=dtype)
def _append_output(p_matches, ts_window, out_arr_group, acc_preconditioner, acc_preconditioned):
    chunk_preconditioner = np.column_stack([
        p_matches,
        np.full_like(p_matches, ts_window[0]),
        np.full_like(p_matches, ts_window[-1])
    ])
    chunk_preconditioned = np.where(np.isin(out_arr_group[:, 0], p_matches))[0]
    if chunk_preconditioner.size:
        acc_preconditioner = np.vstack([acc_preconditioner, chunk_preconditioner])
    if chunk_preconditioned.size:
        acc_preconditioned = np.vstack([acc_preconditioned, chunk_preconditioned])
    return acc_preconditioner, acc_preconditioned

In [ ]:
def MoistPreconditioningSubset(Z,Y,X,type1='CL'): #type1_options = ['CL', 'nonCL', 'SBZ', 'nonSBZ', 'ColdPool']
    global index_adjust

    #READING IN TRACKED PARCEL LISTS
    out_arr_ALL=globals()[f"{type1}_ALL_out_arr"+"_2"].copy()
    out_arr_SHALLOW=globals()[f"{type1}_SHALLOW_out_arr"+"_2"].copy()
    out_arr_DEEP=globals()[f"{type1}_DEEP_out_arr"+"_2"].copy()

    #Initialize Outputs Before the Loop
    out_arr_Preconditioner_SHALLOW, out_arr_Preconditioner_DEEP, \
    out_arr_Preconditioned_SHALLOW, out_arr_Preconditioned_DEEP = \
        (_init_empty(out_arr_ALL.dtype) for _ in range(4))


    #CALCULATING
    for row in range(out_arr_ALL.shape[0]): #Running through All Parcels
        after=out_arr_ALL[row,3]
        p=out_arr_ALL[row,0]
        
        #Getting Time from ALL Parcel Detrainment plus Another 30 Minutes
        ts_end = min(out_arr_ALL[row, 2] + 1 + after, len(data1['time'])) #this takes care of exceeding buffers
        t_end_clip = min(ts_end + int(30 * minutes), len(data1['time']) - 1) #clipping so doesn't go past data end time
        ts_window = np.arange(ts_end + 1, t_end_clip + 1) #looking for detrained moisture entering other parcels within 30 mins
        if ts_window.size == 0: #checking for emptiness
            continue
        
        #Find if Parcel Enters another SHALLOW or DEEP Updraft
        zs_window=Z[ts_window,p-index_adjust]
        ys_window=Y[ts_window,p-index_adjust]
        xs_window=X[ts_window,p-index_adjust]
        
        #Subsetting out Times where No Overlap with Detrained Parcels Occur
        Array_SHALLOW=SubsetTimeOverlap(out_arr_SHALLOW.copy(), ts_window); Array_DEEP=SubsetTimeOverlap(out_arr_DEEP.copy(), ts_window)
        Array_SHALLOW=Array_SHALLOW[Array_SHALLOW[:,0]!=p]; Array_DEEP=Array_DEEP[Array_DEEP[:,0]!=p]
        
        #GETTING ps, t1s, and t2s
        ps_SHALLOW=Array_SHALLOW[:,0]; ps_DEEP=Array_DEEP[:,0]
        t1s_SHALLOW=Array_SHALLOW[:,1]; t1s_DEEP=Array_DEEP[:,1]
        t2s_SHALLOW=Array_SHALLOW[:,2]; t2s_DEEP=Array_DEEP[:,2]

        #Getting list of p,t,Z,Y,X for each SHALLOW/DEEP parcel 
        ZYX_window_ALL = np.vstack([np.ones_like(ts_window)*p, ts_window, zs_window, ys_window, xs_window])
        ZYX_window_SHALLOW = build_ZYX(ps_SHALLOW, t1s_SHALLOW, t2s_SHALLOW, Z, Y, X, index_adjust)
        ZYX_window_DEEP = build_ZYX(ps_DEEP,    t1s_DEEP,    t2s_DEEP,    Z, Y, X, index_adjust)

        #FINDING MATCHES 
        #where parcels from ALL enter other SHALLOW or DEEP trajectories
        p_matches_SHALLOW = match_p_values(ZYX_window_ALL, ZYX_window_SHALLOW)
        p_matches_DEEP = match_p_values(ZYX_window_ALL, ZYX_window_DEEP)

        #FORMING OUTPUT ARRAY (consolidated)
        # SHALLOW
        out_arr_Preconditioner_SHALLOW, out_arr_Preconditioned_SHALLOW = _append_output(
            p_matches_SHALLOW, ts_window, out_arr_SHALLOW,
            out_arr_Preconditioner_SHALLOW, out_arr_Preconditioned_SHALLOW
        )
        # DEEP
        out_arr_Preconditioner_DEEP, out_arr_Preconditioned_DEEP = _append_output(
            p_matches_DEEP, ts_window, out_arr_DEEP,
            out_arr_Preconditioner_DEEP, out_arr_Preconditioned_DEEP
        )

    return {
        f"Preconditioner_SHALLOW_{type1}": out_arr_Preconditioner_SHALLOW,
        f"Preconditioner_DEEP_{type1}": out_arr_Preconditioner_DEEP,
        f"Preconditioned_SHALLOW_{type1}": out_arr_Preconditioned_SHALLOW,
        f"Preconditioned_DEEP_{type1}": out_arr_Preconditioned_DEEP
    }

In [ ]:
def RunCalculation_Simultaneous(Z,Y,X):
    #VARs is a dictionary created by the function MakeDictionary
    Dictionary = {}

    type1_options = ['CL', 'nonCL', 'SBZ', 'nonSBZ', 'ColdPool']

    for t1 in type1_options:
        # print(f"Running for type {t1}")
        Single_Dictionary=MoistPreconditioningSubset(Z,Y,X,type1=t1)
        Dictionary.update(Single_Dictionary)
    return Dictionary

In [ ]:
def SaveData(data_dict, job_id):
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/subsetting/'
    out_file = dir2 + f"parcel_tracking_SUBSET_Preconditioning_{res}_{t_res}_{Np_str}_{job_id}" #*#*#

    # Write the data to HDF5 file
    with h5py.File(out_file, 'w') as h5f:
        for key, value in data_dict.items():
            h5f.create_dataset(key, data=value)

In [ ]:
##############################################
#RUNNING

In [ ]:
#*#* BIG ISSUE: it seems like job_array causes algorithm to miss out of some

In [ ]:
[start_slurm_job,end_slurm_job]=StartSlurmJobArray(num_jobs=num_jobs,num_slurm_jobs=num_slurm_jobs,ISRUN=False) #if ISRUN is False, then will not run using slurm_job_array
print(f"Running on Slurm_Jobs for Slurm_Job_Ids: {(start_slurm_job,end_slurm_job-1)}")

job_id_list=np.arange(start_slurm_job,end_slurm_job)
for job_id in job_id_list:
    if job_id % 1 == 0: print(f'current job_id = {job_id}')
    [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)

    #SLICING DATA
    parcel=parcel1.isel(xh=slice(start_job,end_job))
    apply_job_array_filter(start_job, end_job)

    #GETTING DATA AND PUTTING IN A DICTIONARY
    [Z,Y,X,parcel_z] = GetSpatialData(start_job,end_job) 

    #CALCULATING AND SAVING
    Dictionary=RunCalculation_Simultaneous(Z,Y,X) #VERSION2: EACH VARIABLE SIMULTANEOUSLY (RECOMMENDED)
    SaveData(Dictionary,job_id)
    if any(arr.size > 0 for arr in Dictionary.values())==True: #*#*TESTING
        print(f'non-empty')

    # print_memory_usage("Current Memory")
    if job_id==job_id_list[0]:
        Dictionary_keys=list(Dictionary.keys())
    del Dictionary


In [ ]:
######################################
#RECOMBINING
recombine=False #KEEP FALSE WHEN RUNNING
recombine=True

In [ ]:
def ReadData(var_name,job_id):
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/subsetting/'
    out_file=dir2+f"parcel_tracking_SUBSET_Preconditioning_{res}_{t_res}_{Np_str}_{job_id}"
    with h5py.File(out_file, 'r') as f:
        out = f[var_name][:]
    return out
# ReadData(var_name,job_id)
def SaveFinalData(dict,out_file):    
    with h5py.File(out_file,'w') as f:
        for key in dict:
            print(key)
            f.create_dataset(key, data=dict[key])

In [ ]:
if recombine==True:
    #GETTING COUNTS FOR MAKING INITIAL RECOMBINED ARRAYS LATER
    count_dict = {key: [] for key in Dictionary_keys}
    def MakeCount(count_dict):
        print('Getting Tracked Parcel Count')
        for job_id in np.arange(1,num_jobs+1):
            if job_id % 10==0: print(f"current job_id: {job_id}")
            for key in count_dict:
                data_dict_key=ReadData(key,job_id)
                count_dict[key].append(data_dict_key.shape[0])
        combined_counts={key: sum(counts) for key, counts in count_dict.items()}
        return combined_counts
    combined_counts=MakeCount(count_dict)
    print(combined_counts)

In [ ]:
if recombine==True:
    def MakeEmpty(counts_dict):
        empty_dict = {}
        for key, count in counts_dict.items():
            empty_dict[key] = np.zeros((count, 4), dtype=int)
        return empty_dict
    dict=MakeEmpty(combined_counts)

In [ ]:
if recombine==True:
    for job_id in np.arange(1,num_jobs+1):
        if job_id % 10==0: print(f"current job_id: {job_id}")
            
        for key in dict:
            var=ReadData(key,job_id)
            if var.size!=0:
                a=np.where(np.all(dict[key] == 0, axis=1))[0][0]
                b=a+var.shape[0]
                # print(key,a,b) #TESTING
                dict[key][a:b]=var

In [ ]:
if recombine==True:
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
    out_file=dir2+f"parcel_tracking_SUBSET_Preconditioning_{res}_{t_res}_{Np_str}.h5"
    SaveFinalData(dict,out_file)